# Experiment 1-4
### Objective

#### To determine whether a neural network can predict the population genetic parameter theta (θ) from simulated genetic data using a simple binning approach. This experiment serves as an initial step to assess the feasibility of our methodology before exploring more complex models.

### Methodology

#### Data Simulation

	•	Simulation Tool: msprime library for simulating ancestral histories and mutations.
	•	Number of Simulations: 10,000 datasets.
	•	Population Parameters:
	•	Effective Population Size ($N$): Fixed at 10,000 individuals.
	•	Mutation Rates ($\mu$):
	•	Varied across simulations.
	•	Generated 20 mutation rates logarithmically spaced between $1 \times 10^{-9}$ and $5 \times 10^{-8}$.
	•	Theta Calculation:
	•	Calculated for each simulation using the formula: $\theta = 4N\mu$.

#### Feature Extraction

	•	Genomic Sequence:
	•	Total length: 50,000 base pairs.
	•	Divided into 50 bins, each 1,000 base pairs long.
	•	Mutation Counting:
	•	Counted the number of mutations within each bin for each simulation.
	•	Data Normalization:
	•	Normalized mutation counts within each simulation to scale between 0 and 1.
	•	Ensured that mutation counts are comparable across different simulations.

#### Neural Network Model

	•	Architecture:
	•	Input Layer: Accepts normalized mutation counts per bin, shaped as (50 bins, 1 feature per bin).
	•	Convolutional Layers:
	•	Conv1D Layer 1: 32 filters, kernel size of 3, ReLU activation.
	•	Conv1D Layer 2: 64 filters, kernel size of 3, ReLU activation.
	•	Flatten Layer: Converts the output from convolutional layers into a 1D vector.
	•	Dense Layers:
	•	Dense Layer: 64 neurons, ReLU activation.
	•	Output Layer: 1 neuron for regression output (predicting $\theta$).
	•	Compilation:
	•	Optimizer: Adam optimizer.
	•	Loss Function: Mean Squared Error (MSE).
	•	Metrics: Mean Absolute Error (MAE).
	•	Training Parameters:
	•	Epochs: Maximum of 50 epochs.
	•	Batch Size: 32 samples per batch.
	•	Early Stopping:
	•	Monitors validation loss.
	•	Stops training if no improvement after 5 epochs.
	•	Restores the best model weights.

#### Data Splitting

	•	Training Set: 70% of the data.
	•	Validation Set: 15% of the data.
	•	Test Set: 15% of the data.
	•	Data Shuffling: Ensures random distribution of samples across sets.

#### Note
	The main purpose of Experiment 1 was to test the feasibality of our research goal. In order to achieve this, we constructed simple-straight forward models and evaluated their performance. Since we were trying things out, there are a couple of other models that has not been mentioned in the report. These approaches will be explained briefly, but will not be explained into depth.

## 1 Initial Prototype - Binning approach with only 1 feature(Number of Mutations)
Key Characteristics:

#### Simulation Parameters

	•	Population Size (N): Fixed at 10,000.
	•	Mutation Rate (μ): Varied across simulations, chosen from 20 logarithmically spaced values between $1 \times 10^{-9}$ and $5 \times 10^{-8}$.
	•	Theta Calculation: $\theta = 4N\mu$.

#### Feature Extraction

	•	Mutation Counts per Bin:
	•	The genomic sequence is divided into 50 bins (each 1,000 base pairs).
	•	Counts the number of mutations in each bin.
	•	Mutation counts are normalized within each simulation (scaled between 0 and 1).

#### Model Architecture

	•	Convolutional Neural Network (CNN): Uses Conv1D layers.
	•	Input Shape: (50 bins, 1 feature per bin).
	•	Two Conv1D layers followed by a Flatten layer and Dense layers.
	•	Predicts $\theta$ from the normalized mutation counts per bin.

In [1]:
# Daata preparation, model definition, training, and evaluation using
# 10,000 data points
# Feature -- 1: Number of Mutations
# Binning Approach

import msprime
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random

# Simulation constants
NUM_SAMPLES = 50           # Number of diploid samples per simulation
SEQUENCE_LENGTH = 50_000   # Length of the genomic region in base pairs
BIN_SIZE = 1_000           # Bin size in base pairs
NUM_BINS = SEQUENCE_LENGTH // BIN_SIZE  # Total number of bins
POPULATION_SIZE = 10_000   # Effective population size (N)

# Data generation parameters
NUM_DATASETS = 10_000      # Total number of datasets to simulate

# Generate a range of mutation rates (μ)
MIN_MU = 1e-9
MAX_MU = 5e-8

# Generate mutation rates on a logarithmic scale
mu_values = np.logspace(np.log10(MIN_MU), np.log10(MAX_MU), num=20)

# Compute corresponding theta values
theta_values_list = [4 * POPULATION_SIZE * mu for mu in mu_values]

# Initialize lists to store inputs and labels
inputs = []
labels = []

# For reproducibility
random.seed(42)
np.random.seed(42)

# Loop to generate NUM_DATASETS datasets
for i in range(NUM_DATASETS):
    # Randomly select a mutation rate μ
    mu = random.choice(mu_values)
    theta = 4 * POPULATION_SIZE * mu

    # Simulate ancestral history
    ts = msprime.sim_ancestry(
        samples=NUM_SAMPLES,
        recombination_rate=0,
        sequence_length=SEQUENCE_LENGTH,
        population_size=POPULATION_SIZE,
        random_seed=random.randint(1, 1e6)
    )

    # Simulate mutations
    mts = msprime.sim_mutations(ts, rate=mu, random_seed=random.randint(1, 1e6))

    # Initialize mutation counts per bin
    mutation_counts = np.zeros(NUM_BINS)

    # Process mutations
    for variant in mts.variants():
        position = variant.site.position
        bin_index = int(position // BIN_SIZE)
        mutation_counts[bin_index] += 1

    # Normalize mutation counts
    max_count = mutation_counts.max()
    if max_count > 0:
        mutation_counts_normalized = mutation_counts / max_count
    else:
        mutation_counts_normalized = mutation_counts

    # Reshape for CNN input
    input_cnn = mutation_counts_normalized.reshape((NUM_BINS, 1))  # Shape: (NUM_BINS, 1)

    # Append to lists
    inputs.append(input_cnn)
    labels.append(theta)

    # Optional: Print progress
    if (i + 1) % 500 == 0:
        print(f"Processed {i + 1}/{NUM_DATASETS} datasets")

# Convert lists to NumPy arrays
inputs_array = np.array(inputs)      # Shape: (NUM_DATASETS, NUM_BINS, 1)
labels_array = np.array(labels)      # Shape: (NUM_DATASETS,)

# Shuffle and split the data
X_train_val, X_test, y_train_val, y_test = train_test_split(
    inputs_array, labels_array, test_size=0.15, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.15 / 0.85, random_state=42)

print(f"Training samples: {X_train.shape[0]}")
print(f"Validation samples: {X_val.shape[0]}")
print(f"Test samples: {X_test.shape[0]}")

# Define the CNN model
input_shape = (NUM_BINS, 1)  # (Number of bins, number of features per bin)

model = Sequential([
    Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=input_shape),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1)  # Output layer for regression
])

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

model.summary()

# Early stopping to prevent overfitting
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping]
)

# Evaluate the model on the test set
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"Test Loss (MSE): {test_loss}")
print(f"Test MAE: {test_mae}")

# Plot training & validation loss values
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], label='Training Loss (MSE)')
plt.plot(history.history['val_loss'], label='Validation Loss (MSE)')
plt.title('Model Loss')
plt.ylabel('Loss (MSE)')
plt.xlabel('Epoch')
plt.legend()
plt.show()

# Plot training & validation MAE values
plt.figure(figsize=(10, 5))
plt.plot(history.history['mae'], label='Training MAE')
plt.plot(history.history['val_mae'], label='Validation MAE')
plt.title('Model Mean Absolute Error')
plt.ylabel('MAE')
plt.xlabel('Epoch')
plt.legend()
plt.show()

# Make predictions on the test set
y_pred = model.predict(X_test)

# Plot predicted vs. true theta values
plt.figure(figsize=(8, 8))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')  # Diagonal line
plt.xlabel('True Theta Values')
plt.ylabel('Predicted Theta Values')
plt.title('Predicted vs. True Theta Values')
plt.show()

Processed 500/10000 datasets
Processed 1000/10000 datasets
Processed 1500/10000 datasets
Processed 2000/10000 datasets
Processed 2500/10000 datasets
Processed 3000/10000 datasets
Processed 3500/10000 datasets
Processed 4000/10000 datasets
Processed 4500/10000 datasets
Processed 5000/10000 datasets
Processed 5500/10000 datasets
Processed 6000/10000 datasets
Processed 6500/10000 datasets
Processed 7000/10000 datasets
Processed 7500/10000 datasets
Processed 8000/10000 datasets
Processed 8500/10000 datasets
Processed 9000/10000 datasets
Processed 9500/10000 datasets
Processed 10000/10000 datasets
Training samples: 7000
Validation samples: 1500
Test samples: 1500


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-12-05 18:21:45.565858: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-12-05 18:21:45.565888: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-12-05 18:21:45.565902: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-12-05 18:21:45.566090: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-12-05 18:21:45.566100: I tensorflow/core/common_runtime/pluggable_device/plu

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 48, 32)         │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 46, 64)         │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2944)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │       188,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 194,881 (761.25 KB)

...

 Non-trainable params: 0 (0.00 B)

Epoch 1/50


2024-12-05 18:21:46.096294: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


219/219 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 0.0034 - mae: 0.0241 - val_loss: 6.5456e-07 - val_mae: 5.0890e-04
Epoch 2/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 5.1827e-07 - mae: 5.0141e-04 - val_loss: 4.3836e-07 - val_mae: 4.7101e-04
Epoch 3/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 3.4460e-07 - mae: 4.6459e-04 - val_loss: 3.9807e-07 - val_mae: 4.6376e-04
Epoch 4/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 3.2076e-07 - mae: 4.5772e-04 - val_loss: 3.7472e-07 - val_mae: 4.6830e-04
Epoch 5/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 3.3128e-07 - mae: 4.6634e-04 - val_loss: 3.5950e-07 - val_mae: 4.6004e-04
Epoch 6/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 3.1915e-07 - mae: 4.5814e-04 - val_loss: 3.5296e-07 - val_mae: 4.6029e-04
Epoch 7/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 3.1572e-07 - mae: 4.5190e-04 - val_loss: 3.5213e-07 - val_mae: 4.5368e-04
Epoch 8/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 3.0679e-07 - 

KeyboardInterrupt: 

## 2 Adding features to prototype - Binning approach with 3 features(Number of Mutations) and varying population size & mutation rate
Key Characteristics:

#### Simulation Parameters:
	•	Population Size (N): Varies across simulations.
	•	Randomly chosen from 10 linearly spaced values between 5,000 and 20,000.
	•	Mutation Rate (μ): Varies across simulations.
	•	Randomly chosen from 10 logarithmically spaced values between 1 \times 10^{-9} and 5 \times 10^{-8}.
	•	Theta Calculation: \theta = 4N\mu.
#### Feature Extraction:
	•	Allele Frequency Spectrum (2D Histogram):
	•	Genomic Bins: The genomic sequence is divided into bins (each 1,000 base pairs).
	•	Frequency Bins: Allele frequencies are divided into 10 bins ranging from 0 to 1.
	•	2D Histogram: For each variant, the count is incremented in the corresponding genomic bin and frequency bin, forming a 2D histogram.
	•	Additional Features per Genomic Bin:
	•	Watterson’s Theta (θw): Calculated per genomic bin.
	•	Nucleotide Diversity (π): Calculated per genomic bin.
	•	Combined Features:
	•	The 2D histogram and additional features are concatenated to form a combined feature matrix per simulation.
#### Data Normalization:
	•	Histogram Normalization: Counts in the histogram are normalized per genomic bin.
	•	Global Normalization of Additional Features:
	•	θw and π are normalized across all simulations to ensure consistency.
#### Model Architecture:
	•	Convolutional Neural Network (CNN): Uses Conv2D layers.
	•	Input shape: (number of genomic bins, number of frequency bins + 2, 1).
	•	Two Conv2D layers followed by a Flatten layer and Dense layers.
	•	Predicts θ from the combined features.
#### Data Shapes:
	•	Inputs Array Shape: (number of datasets, number of genomic bins, number of frequency bins + 2, 1).
#### Additional Analysis:
	•	Population Size and Mutation Rate in Test Results:
	•	The test results include N and μ for each sample.
#### Visualization:
	•	Plots of predicted vs. true θ values, colored by population size or mutation rate.

In [ ]:
import msprime
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import gc
import allel  # scikit-allel library
import pandas as pd

# Simulation constants
NUM_SAMPLES_PER_SIM = 50      # Number of diploid samples per simulation
SEQUENCE_LENGTH = 50_000      # Length of the genomic region in base pairs
BIN_SIZE = 1_000              # Bin size in base pairs
NUM_GENOMIC_BINS = SEQUENCE_LENGTH // BIN_SIZE  # Total number of genomic bins

# Frequency histogram parameters
NUM_FREQUENCY_BINS = 10       # Number of frequency bins
FREQUENCY_BIN_EDGES = np.linspace(0, 1, NUM_FREQUENCY_BINS + 1)  # Edges from 0 to 1 inclusive

# Data generation parameters
NUM_DATASETS = 100000           # Total number of datasets to simulate (adjust as needed)
BATCH_SIZE = 32               # Batch size for training

# Generate a range of population sizes (N)
MIN_N = 5_000
MAX_N = 20_000
N_values = np.linspace(MIN_N, MAX_N, num=10, dtype=int)  # 10 values from 5,000 to 20,000

# Generate a range of mutation rates (μ)
MIN_MU = 1e-9
MAX_MU = 5e-8
mu_values = np.logspace(np.log10(MIN_MU), np.log10(MAX_MU), num=10)  # 10 values on log scale

# Initialize lists to store inputs and labels
inputs = []
labels = []
theta_values = []        # To store theta values
population_sizes = []    # To store N values
mutation_rates = []      # To store μ values

# For reproducibility
random.seed(42)
np.random.seed(42)

# Initialize lists for global normalization
all_theta_w_bins = []
all_pi_bins = []

# Loop to generate NUM_DATASETS datasets
for i in range(NUM_DATASETS):
    # Randomly select a population size N and mutation rate μ
    N = random.choice(N_values)
    mu = random.choice(mu_values)
    theta = 4 * N * mu  # θ = 4Nμ

    # Simulate ancestral history
    ts = msprime.sim_ancestry(
        samples=NUM_SAMPLES_PER_SIM,
        recombination_rate=0,
        sequence_length=SEQUENCE_LENGTH,
        population_size=N,
        random_seed=random.randint(1, 1e6)
    )

    # Simulate mutations
    mts = msprime.sim_mutations(ts, rate=mu, random_seed=random.randint(1, 1e6))

    # Initialize the input matrix for this sample
    input_matrix = np.zeros((NUM_GENOMIC_BINS, NUM_FREQUENCY_BINS), dtype=np.float32)

    # Initialize arrays for additional features
    theta_w_bins = np.zeros(NUM_GENOMIC_BINS, dtype=np.float32)
    pi_bins = np.zeros(NUM_GENOMIC_BINS, dtype=np.float32)

    # Collect genotype data and positions
    genotypes_list = []
    positions_list = []

    # Total number of alleles (diploid samples)
    NUM_ALLELES = ts.num_samples  # Each sample represents one chromosome

    for variant in mts.variants():
        position = variant.site.position
        genomic_bin_index = int(position // BIN_SIZE)

        # Compute frequency
        genotypes = variant.genotypes  # Array of 0s and 1s
        derived_allele_count = np.count_nonzero(genotypes)  # Number of '1's
        frequency = derived_allele_count / NUM_ALLELES  # Allele frequency

        # Determine frequency bin
        freq_bin_index = np.digitize(frequency, FREQUENCY_BIN_EDGES) - 1
        freq_bin_index = min(freq_bin_index, NUM_FREQUENCY_BINS - 1)  # Ensure index is within bounds

        # Increment the count
        input_matrix[genomic_bin_index, freq_bin_index] += 1

        # Collect data for additional features
        genotypes_list.append(genotypes)
        positions_list.append(position)

    # Convert lists to arrays
    if genotypes_list:
        genotypes_array = np.vstack(genotypes_list)  # Shape: (num_variants, NUM_ALLELES)
        positions_array = np.array(positions_list)
    else:
        genotypes_array = np.empty((0, NUM_ALLELES), dtype=np.int8)
        positions_array = np.empty(0)

    # Process each genomic bin for additional features
    for bin_index in range(NUM_GENOMIC_BINS):
        start = bin_index * BIN_SIZE
        end = start + BIN_SIZE
        in_bin = (positions_array >= start) & (positions_array < end)
        genotypes_bin = genotypes_array[in_bin]

        if genotypes_bin.size > 0:
            # Convert genotypes to allele counts per variant
            allele_counts = genotypes_bin.sum(axis=1)

            # Number of segregating sites (S)
            S = allele_counts.size

            # Watterson's Theta (θw)
            a1 = np.sum(1.0 / np.arange(1, NUM_ALLELES))
            theta_w = S / a1 if a1 > 0 else 0.0
            theta_w_bins[bin_index] = theta_w

            # Nucleotide Diversity (π)
            p = allele_counts / NUM_ALLELES
            pi = 2 * np.sum(p * (1 - p))
            pi_bins[bin_index] = pi
        else:
            theta_w_bins[bin_index] = 0.0
            pi_bins[bin_index] = 0.0

    # Collect theta_w_bins and pi_bins for global normalization
    all_theta_w_bins.append(theta_w_bins)
    all_pi_bins.append(pi_bins)

    # Normalize histograms per genomic bin
    bin_sums = input_matrix.sum(axis=1, keepdims=True)
    input_matrix_normalized = input_matrix / bin_sums
    input_matrix_normalized = np.nan_to_num(input_matrix_normalized)  # Replace NaNs with zero

    # We will normalize additional features later

    # Combine features (without normalizing additional features yet)
    additional_features = np.stack([theta_w_bins, pi_bins], axis=1)  # Shape: (NUM_GENOMIC_BINS, 2)
    combined_features = np.concatenate([input_matrix_normalized, additional_features], axis=1)  # Shape: (NUM_GENOMIC_BINS, NUM_FREQUENCY_BINS + 2)

    # Expand dimensions to add channel for CNN input
    input_cnn = combined_features[..., np.newaxis]  # Shape: (NUM_GENOMIC_BINS, NUM_FREQUENCY_BINS + 2, 1)

    # Append to lists
    inputs.append(input_cnn)
    labels.append(theta)
    theta_values.append(theta)
    population_sizes.append(N)
    mutation_rates.append(mu)

    # Optional: Print progress
    if (i + 1) % 100 == 0:
        print(f"Processed {i + 1}/{NUM_DATASETS} datasets")

# Convert lists to NumPy arrays
inputs_array = np.array(inputs, dtype=np.float32)    # Shape: (NUM_DATASETS, NUM_GENOMIC_BINS, NUM_FREQUENCY_BINS + 2, 1)
labels_array = np.array(labels, dtype=np.float32)    # Shape: (NUM_DATASETS,)
population_sizes_array = np.array(population_sizes, dtype=np.int32)
mutation_rates_array = np.array(mutation_rates, dtype=np.float32)

# Clean up
del inputs
del labels
gc.collect()

# Global normalization of additional features
all_theta_w_bins_array = np.concatenate(all_theta_w_bins)
all_pi_bins_array = np.concatenate(all_pi_bins)

global_max_theta_w = np.max(all_theta_w_bins_array)
global_max_pi = np.max(all_pi_bins_array)

# Avoid division by zero
global_max_theta_w = global_max_theta_w if global_max_theta_w > 0 else 1
global_max_pi = global_max_pi if global_max_pi > 0 else 1

# Normalize additional features across the entire dataset
for i in range(NUM_DATASETS):
    # Normalize theta_w_bins and pi_bins in inputs_array
    inputs_array[i, :, NUM_FREQUENCY_BINS, 0] /= global_max_theta_w
    inputs_array[i, :, NUM_FREQUENCY_BINS + 1, 0] /= global_max_pi

# Shuffle and split the data
from sklearn.model_selection import train_test_split

X_train_val, X_test, y_train_val, y_test, N_train_val, N_test = train_test_split(
    inputs_array, labels_array, population_sizes_array, test_size=0.15, random_state=42)

X_train, X_val, y_train, y_val, N_train, N_val = train_test_split(
    X_train_val, y_train_val, N_train_val, test_size=0.15 / 0.85, random_state=42)

print(f"Training samples: {X_train.shape[0]}")
print(f"Validation samples: {X_val.shape[0]}")
print(f"Test samples: {X_test.shape[0]}")

# Clean up
del inputs_array
del labels_array
del population_sizes_array
gc.collect()

# Define the CNN model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense

input_shape = (NUM_GENOMIC_BINS, NUM_FREQUENCY_BINS + 2, 1)  # (Height, Width, Channels)

model = Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1)  # Output layer for regression
])

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

model.summary()

# Early stopping to prevent overfitting
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
val_dataset = val_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# Train the model
history = model.fit(
    train_dataset,
    epochs=50,
    validation_data=val_dataset,
    callbacks=[early_stopping]
)

# Evaluate the model on the test set
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

test_loss, test_mae = model.evaluate(test_dataset)
print(f"Test Loss (MSE): {test_loss}")
print(f"Test MAE: {test_mae}")

# Plot training & validation loss values
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], label='Training Loss (MSE)')
plt.plot(history.history['val_loss'], label='Validation Loss (MSE)')
plt.title('Model Loss')
plt.ylabel('Loss (MSE)')
plt.xlabel('Epoch')
plt.legend()
plt.show()

# Plot training & validation MAE values
plt.figure(figsize=(10, 5))
plt.plot(history.history['mae'], label='Training MAE')
plt.plot(history.history['val_mae'], label='Validation MAE')
plt.title('Model Mean Absolute Error')
plt.ylabel('MAE')
plt.xlabel('Epoch')
plt.legend()
plt.show()

# Make predictions on the test set
y_pred = model.predict(test_dataset)
y_pred = y_pred.flatten()

# Optional: Include population sizes and mutation rates for the test set
N_test = N_test[:len(y_test)]
mu_test = mutation_rates_array[-len(y_test):]

# Create a DataFrame with test results
test_results = pd.DataFrame({
    'True Theta': y_test,
    'Predicted Theta': y_pred,
    'Population Size': N_test,
    'Mutation Rate': mu_test
})

# Plot predicted vs. true theta values
plt.figure(figsize=(8, 8))
plt.scatter(test_results['True Theta'], test_results['Predicted Theta'], alpha=0.5)
plt.plot([test_results['True Theta'].min(), test_results['True Theta'].max()],
         [test_results['True Theta'].min(), test_results['True Theta'].max()], 'r--')
plt.xlabel('True Theta Values')
plt.ylabel('Predicted Theta Values')
plt.title('Predicted vs. True Theta Values')
plt.show()

# Optional Analysis: Plot predictions colored by Population Size
plt.figure(figsize=(8, 8))
scatter = plt.scatter(test_results['True Theta'], test_results['Predicted Theta'],
                      c=test_results['Population Size'], cmap='viridis', alpha=0.7)
plt.plot([test_results['True Theta'].min(), test_results['True Theta'].max()],
         [test_results['True Theta'].min(), test_results['True Theta'].max()], 'r--')
plt.xlabel('True Theta Values')
plt.ylabel('Predicted Theta Values')
plt.title('Predicted vs. True Theta Values (Colored by Population Size)')
cbar = plt.colorbar(scatter)
cbar.set_label('Population Size')
plt.show()

# Optional Analysis: Plot predictions colored by Mutation Rate
plt.figure(figsize=(8, 8))
scatter = plt.scatter(test_results['True Theta'], test_results['Predicted Theta'],
                      c=test_results['Mutation Rate'], cmap='plasma', alpha=0.7)
plt.plot([test_results['True Theta'].min(), test_results['True Theta'].max()],
         [test_results['True Theta'].min(), test_results['True Theta'].max()], 'r--')
plt.xlabel('True Theta Values')
plt.ylabel('Predicted Theta Values')
plt.title('Predicted vs. True Theta Values (Colored by Mutation Rate)')
cbar = plt.colorbar(scatter)
cbar.set_label('Mutation Rate')
plt.show()

## 3 Additional feature - Including Allele Frequency Spectrum

Key Characteristics:

#### Simulation Parameters

	•	Population Size (N): Varies across simulations.
	•	Randomly selected from 10 linearly spaced values between 5,000 and 20,000.
	•	Mutation Rate (μ): Varies across simulations.
	•	Randomly selected from 10 logarithmically spaced values between 1 \times 10^{-9} and 5 \times 10^{-8}.
	•	Theta Calculation: \theta = 4N\mu.
	•	Number of Simulations: 100,000 datasets.
	•	Samples per Simulation: 50 diploid individuals (100 chromosomes).
	•	Sequence Length: 50,000 base pairs.
	•	Bin Size: 1,000 base pairs, resulting in 50 genomic bins.

#### Feature Extraction

##### Allele Frequency Spectrum (2D Histogram)

	•	Genomic Bins:
	•	The genomic sequence is divided into 50 bins, each 1,000 base pairs long.
	•	Frequency Bins:
	•	Allele frequencies are divided into 10 bins ranging from 0 to 1.
	•	Frequency bin edges are defined using np.linspace(0, 1, NUM_FREQUENCY_BINS + 1).
	•	2D Histogram:
	•	For each variant (mutation), counts are incremented in the corresponding genomic bin and frequency bin, forming a 2D histogram.
	•	This results in an input matrix of shape (50 genomic bins, 10 frequency bins).

##### Additional Features per Genomic Bin

	•	Watterson’s Theta (θw):
	•	Calculated per genomic bin using the number of segregating sites (S) and the harmonic sum
	•	Nucleotide Diversity (π):
	•	Calculated per genomic bin based on allele frequencies.

#### Data Normalization
	•	Histogram Normalization:
	•	Counts in the histogram are normalized per genomic bin by dividing each frequency bin count by the total count of mutations in that genomic bin.
	•	This handles varying mutation counts across bins and simulations.
	•	NaN values resulting from division by zero are replaced with zeros.
	•	Global Normalization of Additional Features:
	•	θw and π are concatenated across all datasets to find the global maximum values.
	•	Each θw and π value is then divided by its global maximum to scale these features between 0 and 1.
	•	This ensures consistency across the entire dataset.

#### Combined Features
	•	Feature Matrix:
	•	The normalized histogram and the normalized additional features (θw and π) are concatenated along the feature axis.
	•	Results in a combined feature matrix of shape (50 genomic bins, 12 features per bin), where:
	•	10 features from frequency bins.
	•	2 features from θw and π.
	•	Input for the Model:
	•	The combined feature matrix is used as input to the model without adding extra dimensions, suitable for RNN input.
	•	Input Shape: (50 genomic bins, 12 features per bin).

#### Model Architecture

Recurrent Neural Network (RNN) with LSTM Layer

	•	Input Layer:
	•	Accepts a sequence of genomic bins, each represented by a 12-dimensional feature vector.
	•	Input Shape: (50 genomic bins, 12 features per bin).
	•	LSTM Layer:
	•	Units: 64.
	•	Function: Processes the sequential data to capture dependencies and patterns across genomic bins.
	•	Return Sequences: False (only the final output is used).
	•	Dense Layers:
	•	Hidden Dense Layer:
	•	64 neurons with ReLU activation.
	•	Output Layer:
	•	1 neuron for regression output (predicting θ).

Compilation

	•	Optimizer: Adam optimizer.
	•	Loss Function: Mean Squared Error (MSE), suitable for regression tasks.
	•	Metrics: Mean Absolute Error (MAE) for additional performance evaluation.

Training Parameters

	•	Epochs: Maximum of 50 epochs.
	•	Batch Size: 32 samples per batch.
	•	Early Stopping:
	•	Monitors validation loss.
	•	Stops training if no improvement after 5 epochs (patience).
	•	Restores the best model weights to prevent overfitting.

In [ ]:
import msprime
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import gc
import allel  # scikit-allel library
import pandas as pd

# Simulation constants
NUM_SAMPLES_PER_SIM = 50      # Number of diploid samples per simulation
SEQUENCE_LENGTH = 50_000      # Length of the genomic region in base pairs
BIN_SIZE = 1_000              # Bin size in base pairs
NUM_GENOMIC_BINS = SEQUENCE_LENGTH // BIN_SIZE  # Total number of genomic bins

# Frequency histogram parameters
NUM_FREQUENCY_BINS = 10       # Number of frequency bins
FREQUENCY_BIN_EDGES = np.linspace(0, 1, NUM_FREQUENCY_BINS + 1)  # Edges from 0 to 1 inclusive

# Data generation parameters
NUM_DATASETS = 100000           # Total number of datasets to simulate (adjust as needed)
BATCH_SIZE = 32               # Batch size for training

# Generate a range of population sizes (N)
MIN_N = 5_000
MAX_N = 20_000
N_values = np.linspace(MIN_N, MAX_N, num=10, dtype=int)  # 10 values from 5,000 to 20,000

# Generate a range of mutation rates (μ)
MIN_MU = 1e-9
MAX_MU = 5e-8
mu_values = np.logspace(np.log10(MIN_MU), np.log10(MAX_MU), num=10)  # 10 values on log scale

# Initialize lists to store inputs and labels
inputs = []
labels = []
theta_values = []        # To store theta values
population_sizes = []    # To store N values
mutation_rates = []      # To store μ values

# For reproducibility
random.seed(42)
np.random.seed(42)

# Initialize lists for global normalization
all_theta_w_bins = []
all_pi_bins = []

# Loop to generate NUM_DATASETS datasets
for i in range(NUM_DATASETS):
    # Randomly select a population size N and mutation rate μ
    N = random.choice(N_values)
    mu = random.choice(mu_values)
    theta = 4 * N * mu  # θ = 4Nμ

    # Simulate ancestral history
    ts = msprime.sim_ancestry(
        samples=NUM_SAMPLES_PER_SIM,
        recombination_rate=0,
        sequence_length=SEQUENCE_LENGTH,
        population_size=N,
        random_seed=random.randint(1, 1e6)
    )

    # Simulate mutations
    mts = msprime.sim_mutations(ts, rate=mu, random_seed=random.randint(1, 1e6))

    # Initialize the input matrix for this sample
    input_matrix = np.zeros((NUM_GENOMIC_BINS, NUM_FREQUENCY_BINS), dtype=np.float32)

    # Initialize arrays for additional features
    theta_w_bins = np.zeros(NUM_GENOMIC_BINS, dtype=np.float32)
    pi_bins = np.zeros(NUM_GENOMIC_BINS, dtype=np.float32)

    # Collect genotype data and positions
    genotypes_list = []
    positions_list = []

    # Total number of alleles (diploid samples)
    NUM_ALLELES = ts.num_samples  # Each sample represents one chromosome

    for variant in mts.variants():
        position = variant.site.position
        genomic_bin_index = int(position // BIN_SIZE)

        # Compute frequency
        genotypes = variant.genotypes  # Array of 0s and 1s
        derived_allele_count = np.count_nonzero(genotypes)  # Number of '1's
        frequency = derived_allele_count / NUM_ALLELES  # Allele frequency

        # Determine frequency bin
        freq_bin_index = np.digitize(frequency, FREQUENCY_BIN_EDGES) - 1
        freq_bin_index = min(freq_bin_index, NUM_FREQUENCY_BINS - 1)  # Ensure index is within bounds

        # Increment the count
        input_matrix[genomic_bin_index, freq_bin_index] += 1

        # Collect data for additional features
        genotypes_list.append(genotypes)
        positions_list.append(position)

    # Convert lists to arrays
    if genotypes_list:
        genotypes_array = np.vstack(genotypes_list)  # Shape: (num_variants, NUM_ALLELES)
        positions_array = np.array(positions_list)
    else:
        genotypes_array = np.empty((0, NUM_ALLELES), dtype=np.int8)
        positions_array = np.empty(0)

    # Process each genomic bin for additional features
    for bin_index in range(NUM_GENOMIC_BINS):
        start = bin_index * BIN_SIZE
        end = start + BIN_SIZE
        in_bin = (positions_array >= start) & (positions_array < end)
        genotypes_bin = genotypes_array[in_bin]

        if genotypes_bin.size > 0:
            # Convert genotypes to allele counts per variant
            allele_counts = genotypes_bin.sum(axis=1)

            # Number of segregating sites (S)
            S = allele_counts.size

            # Watterson's Theta (θw)
            a1 = np.sum(1.0 / np.arange(1, NUM_ALLELES))
            theta_w = S / a1 if a1 > 0 else 0.0
            theta_w_bins[bin_index] = theta_w

            # Nucleotide Diversity (π)
            p = allele_counts / NUM_ALLELES
            pi = 2 * np.sum(p * (1 - p))
            pi_bins[bin_index] = pi
        else:
            theta_w_bins[bin_index] = 0.0
            pi_bins[bin_index] = 0.0

    # Collect theta_w_bins and pi_bins for global normalization
    all_theta_w_bins.append(theta_w_bins)
    all_pi_bins.append(pi_bins)

    # Normalize histograms per genomic bin
    bin_sums = input_matrix.sum(axis=1, keepdims=True)
    input_matrix_normalized = input_matrix / bin_sums
    input_matrix_normalized = np.nan_to_num(input_matrix_normalized)  # Replace NaNs with zero

    # We will normalize additional features later

    # Combine features (without normalizing additional features yet)
    additional_features = np.stack([theta_w_bins, pi_bins], axis=1)  # Shape: (NUM_GENOMIC_BINS, 2)
    combined_features = np.concatenate([input_matrix_normalized, additional_features], axis=1)  # Shape: (NUM_GENOMIC_BINS, NUM_FREQUENCY_BINS + 2)

    # No need to add channel dimension for RNNs
    input_rnn = combined_features  # Shape: (NUM_GENOMIC_BINS, NUM_FREQUENCY_BINS + 2)

    # Append to lists
    inputs.append(input_rnn)
    labels.append(theta)
    theta_values.append(theta)
    population_sizes.append(N)
    mutation_rates.append(mu)

    # Optional: Print progress
    if (i + 1) % 100 == 0:
        print(f"Processed {i + 1}/{NUM_DATASETS} datasets")

# Convert lists to NumPy arrays
inputs_array = np.array(inputs, dtype=np.float32)    # Shape: (NUM_DATASETS, NUM_GENOMIC_BINS, NUM_FREQUENCY_BINS + 2)
labels_array = np.array(labels, dtype=np.float32)    # Shape: (NUM_DATASETS,)
population_sizes_array = np.array(population_sizes, dtype=np.int32)
mutation_rates_array = np.array(mutation_rates, dtype=np.float32)

# Clean up
del inputs
del labels
gc.collect()

# Global normalization of additional features
all_theta_w_bins_array = np.concatenate(all_theta_w_bins)
all_pi_bins_array = np.concatenate(all_pi_bins)

global_max_theta_w = np.max(all_theta_w_bins_array)
global_max_pi = np.max(all_pi_bins_array)

# Avoid division by zero
global_max_theta_w = global_max_theta_w if global_max_theta_w > 0 else 1
global_max_pi = global_max_pi if global_max_pi > 0 else 1

# Normalize additional features across the entire dataset
for i in range(NUM_DATASETS):
    # Normalize theta_w_bins and pi_bins in inputs_array
    inputs_array[i, :, NUM_FREQUENCY_BINS] /= global_max_theta_w
    inputs_array[i, :, NUM_FREQUENCY_BINS + 1] /= global_max_pi

# Shuffle and split the data
from sklearn.model_selection import train_test_split

X_train_val, X_test, y_train_val, y_test, N_train_val, N_test = train_test_split(
    inputs_array, labels_array, population_sizes_array, test_size=0.15, random_state=42)

X_train, X_val, y_train, y_val, N_train, N_val = train_test_split(
    X_train_val, y_train_val, N_train_val, test_size=0.15 / 0.85, random_state=42)

print(f"Training samples: {X_train.shape[0]}")
print(f"Validation samples: {X_val.shape[0]}")
print(f"Test samples: {X_test.shape[0]}")

# Clean up
del inputs_array
del labels_array
del population_sizes_array
gc.collect()

# Define the RNN model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

input_shape = (NUM_GENOMIC_BINS, NUM_FREQUENCY_BINS + 2)  # (Timesteps, Features)

model = Sequential([
    LSTM(units=64, return_sequences=False, input_shape=input_shape),
    Dense(64, activation='relu'),
    Dense(1)  # Output layer for regression
])

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

model.summary()

# Early stopping to prevent overfitting
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
val_dataset = val_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# Train the model
history = model.fit(
    train_dataset,
    epochs=50,
    validation_data=val_dataset,
    callbacks=[early_stopping]
)

# Evaluate the model on the test set
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

test_loss, test_mae = model.evaluate(test_dataset)
print(f"Test Loss (MSE): {test_loss}")
print(f"Test MAE: {test_mae}")

# Plot training & validation loss values
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], label='Training Loss (MSE)')
plt.plot(history.history['val_loss'], label='Validation Loss (MSE)')
plt.title('Model Loss')
plt.ylabel('Loss (MSE)')
plt.xlabel('Epoch')
plt.legend()
plt.show()

# Plot training & validation MAE values
plt.figure(figsize=(10, 5))
plt.plot(history.history['mae'], label='Training MAE')
plt.plot(history.history['val_mae'], label='Validation MAE')
plt.title('Model Mean Absolute Error')
plt.ylabel('MAE')
plt.xlabel('Epoch')
plt.legend()
plt.show()

# Make predictions on the test set
y_pred = model.predict(test_dataset)
y_pred = y_pred.flatten()

# Optional: Include population sizes and mutation rates for the test set
N_test = N_test[:len(y_test)]
mu_test = mutation_rates_array[-len(y_test):]

# Create a DataFrame with test results
test_results = pd.DataFrame({
    'True Theta': y_test,
    'Predicted Theta': y_pred,
    'Population Size': N_test,
    'Mutation Rate': mu_test
})

# Plot predicted vs. true theta values
plt.figure(figsize=(8, 8))
plt.scatter(test_results['True Theta'], test_results['Predicted Theta'], alpha=0.5)
plt.plot([test_results['True Theta'].min(), test_results['True Theta'].max()],
         [test_results['True Theta'].min(), test_results['True Theta'].max()], 'r--')
plt.xlabel('True Theta Values')
plt.ylabel('Predicted Theta Values')
plt.title('Predicted vs. True Theta Values')
plt.show()

# Optional Analysis: Plot predictions colored by Population Size
plt.figure(figsize=(8, 8))
scatter = plt.scatter(test_results['True Theta'], test_results['Predicted Theta'],
                      c=test_results['Population Size'], cmap='viridis', alpha=0.7)
plt.plot([test_results['True Theta'].min(), test_results['True Theta'].max()],
         [test_results['True Theta'].min(), test_results['True Theta'].max()], 'r--')
plt.xlabel('True Theta Values')
plt.ylabel('Predicted Theta Values')
plt.title('Predicted vs. True Theta Values (Colored by Population Size)')
cbar = plt.colorbar(scatter)
cbar.set_label('Population Size')
plt.show()

# Optional Analysis: Plot predictions colored by Mutation Rate
plt.figure(figsize=(8, 8))
scatter = plt.scatter(test_results['True Theta'], test_results['Predicted Theta'],
                      c=test_results['Mutation Rate'], cmap='plasma', alpha=0.7)
plt.plot([test_results['True Theta'].min(), test_results['True Theta'].max()],
         [test_results['True Theta'].min(), test_results['True Theta'].max()], 'r--')
plt.xlabel('True Theta Values')
plt.ylabel('Predicted Theta Values')
plt.title('Predicted vs. True Theta Values (Colored by Mutation Rate)')
cbar = plt.colorbar(scatter)
cbar.set_label('Mutation Rate')
plt.show()

## 4 Same features as 3, but different model architecture

Key Characteristics:

#### Simulation Parameters

	•	Population Size (N): Varies across simulations.
	•	Randomly selected from 10 linearly spaced values between 5,000 and 20,000.
	•	Mutation Rate (μ): Varies across simulations.
	•	Randomly selected from 10 logarithmically spaced values between 1 \times 10^{-9} and 5 \times 10^{-8}.
	•	Theta Calculation: \theta = 4N\mu.
	•	Number of Simulations: 1,000,000 datasets.
	•	Samples per Simulation: 50 diploid individuals (100 chromosomes).
	•	Sequence Length: 50,000 base pairs.

#### Feature Extraction

##### Site Frequency Spectrum (SFS)

	•	Allele Frequencies:
	•	For each mutation, the allele frequency is calculated as the proportion of derived alleles among all alleles.
	•	Allele frequencies are collected across the entire sequence.
	•	Frequency Bins:
	•	Allele frequencies are divided into 20 bins ranging from 0 to 1.
	•	Frequency bin edges are defined using np.linspace(0, 1, NUM_FREQUENCY_BINS + 1).
	•	SFS Calculation:
	•	A histogram of allele frequencies is created, resulting in the Site Frequency Spectrum.
	•	The SFS counts are normalized to sum to 1, forming a probability distribution.
	•	Feature Vector: The normalized SFS is represented as a vector of length 20.

##### Additional Global Features

	•	Watterson’s Theta (θw):
	•	Calculated over the entire sequence using the number of segregating sites (S) and the harmonic sum

    •	Nucleotide Diversity (π):
	•	Calculated over the entire sequence based on allele frequencies.

    •	Feature Combination:
	•	The additional features θw and π are appended to the normalized SFS vector.
	•	Total Feature Vector Length: 22 features (20 from SFS + θw + π).

#### Data Normalization
	•	Global Normalization of Additional Features:
	•	θw and π values are collected across all datasets to find their global maximums.
	•	Each θw and π value is then divided by its global maximum to scale these features between 0 and 1.
	•	This ensures consistency across the entire dataset.

#### Model Architecture

##### Multilayer Perceptron (MLP)

	•	Input Layer:
	•	Accepts the combined feature vector for each simulation.
	•	Input Shape: (22 features,)
	•	Hidden Layers:
	•	First Dense Layer:
	•	64 neurons with ReLU activation.
	•	Dropout Layer:
	•	Dropout rate of 0.2 to prevent overfitting.
	•	Second Dense Layer:
	•	64 neurons with ReLU activation.
	•	Dropout Layer:
	•	Dropout rate of 0.2.
	•	Output Layer:
	•	1 neuron for regression output (predicting θ).

##### Compilation

	•	Optimizer: Adam optimizer.
	•	Loss Function: Mean Squared Error (MSE), suitable for regression tasks.
	•	Metrics: Mean Absolute Error (MAE) for additional performance evaluation.

##### Training Parameters

	•	Epochs: Maximum of 100 epochs.
	•	Batch Size: 32 samples per batch.
	•	Early Stopping:
	•	Monitors validation loss.
	•	Stops training if no improvement after 5 epochs (patience).
	•	Restores the best model weights to prevent overfitting.

In [ ]:
import msprime
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import gc
import allel  # scikit-allel library
import pandas as pd

# Simulation constants
NUM_SAMPLES_PER_SIM = 50      # Number of diploid samples per simulation
SEQUENCE_LENGTH = 50_000      # Length of the genomic region in base pairs

# Frequency histogram parameters
NUM_FREQUENCY_BINS = 20       # Number of frequency bins for the SFS
FREQUENCY_BIN_EDGES = np.linspace(0, 1, NUM_FREQUENCY_BINS + 1)  # Edges from 0 to 1 inclusive

# Data generation parameters
NUM_DATASETS = 1000000           # Total number of datasets to simulate (adjust as needed)
BATCH_SIZE = 32               # Batch size for training

# Generate a range of population sizes (N)
MIN_N = 5_000
MAX_N = 20_000
N_values = np.linspace(MIN_N, MAX_N, num=10, dtype=int)  # 10 values from 5,000 to 20,000

# Generate a range of mutation rates (μ)
MIN_MU = 1e-9
MAX_MU = 5e-8
mu_values = np.logspace(np.log10(MIN_MU), np.log10(MAX_MU), num=10)  # 10 values on log scale

# Initialize lists to store inputs and labels
inputs = []
labels = []
theta_values = []        # To store theta values
population_sizes = []    # To store N values
mutation_rates = []      # To store μ values

# For reproducibility
random.seed(42)
np.random.seed(42)

# Initialize lists for global normalization
all_theta_w = []
all_pi = []

# Loop to generate NUM_DATASETS datasets
for i in range(NUM_DATASETS):
    # Randomly select a population size N and mutation rate μ
    N = random.choice(N_values)
    mu = random.choice(mu_values)
    theta = 4 * N * mu  # θ = 4Nμ

    # Simulate ancestral history
    ts = msprime.sim_ancestry(
        samples=NUM_SAMPLES_PER_SIM,
        recombination_rate=0,
        sequence_length=SEQUENCE_LENGTH,
        population_size=N,
        random_seed=random.randint(1, 1e6)
    )

    # Simulate mutations
    mts = msprime.sim_mutations(ts, rate=mu, random_seed=random.randint(1, 1e6))

    # Collect allele frequencies
    allele_frequencies = []

    # Collect genotype data
    genotypes_list = []
    for variant in mts.variants():
        genotypes = variant.genotypes  # Array of 0s and 1s
        derived_allele_count = np.count_nonzero(genotypes)  # Number of '1's
        frequency = derived_allele_count / mts.num_samples  # Allele frequency
        allele_frequencies.append(frequency)

        genotypes_list.append(genotypes)

    # Convert allele frequencies to numpy array
    allele_frequencies = np.array(allele_frequencies)

    # Compute SFS (Site Frequency Spectrum)
    sfs_counts, _ = np.histogram(allele_frequencies, bins=FREQUENCY_BIN_EDGES)
    sfs_normalized = sfs_counts / sfs_counts.sum() if sfs_counts.sum() > 0 else sfs_counts  # Normalize SFS to sum to 1

    # Compute Watterson's Theta (θw) over the entire sequence
    S = len(allele_frequencies)  # Number of segregating sites
    a1 = np.sum(1.0 / np.arange(1, mts.num_samples))
    theta_w = S / a1 if a1 > 0 else 0.0

    # Compute nucleotide diversity (π) over the entire sequence
    if genotypes_list:
        genotypes_array = np.vstack(genotypes_list)  # Shape: (num_variants, NUM_SAMPLES)
        allele_counts = genotypes_array.sum(axis=1)
        p = allele_counts / mts.num_samples
        pi = 2 * np.sum(p * (1 - p))
    else:
        pi = 0.0

    # Collect θw and π for global normalization
    all_theta_w.append(theta_w)
    all_pi.append(pi)

    # Combine features
    # Features: SFS + θw + π
    additional_features = np.array([theta_w, pi], dtype=np.float32)
    combined_features = np.concatenate([sfs_normalized, additional_features])

    # Append to lists
    inputs.append(combined_features)
    labels.append(theta)
    theta_values.append(theta)
    population_sizes.append(N)
    mutation_rates.append(mu)

    # Optional: Print progress
    if (i + 1) % 100 == 0:
        print(f"Processed {i + 1}/{NUM_DATASETS} datasets")

# Convert lists to NumPy arrays
inputs_array = np.array(inputs, dtype=np.float32)    # Shape: (NUM_DATASETS, NUM_FREQUENCY_BINS + 2)
labels_array = np.array(labels, dtype=np.float32)    # Shape: (NUM_DATASETS,)
population_sizes_array = np.array(population_sizes, dtype=np.int32)
mutation_rates_array = np.array(mutation_rates, dtype=np.float32)

# Clean up
del inputs
del labels
gc.collect()

# Convert lists to arrays
all_theta_w_array = np.array(all_theta_w, dtype=np.float32)
all_pi_array = np.array(all_pi, dtype=np.float32)

# Global normalization of θw and π
global_max_theta_w = np.max(all_theta_w_array)
global_max_pi = np.max(all_pi_array)

# Avoid division by zero
global_max_theta_w = global_max_theta_w if global_max_theta_w > 0 else 1
global_max_pi = global_max_pi if global_max_pi > 0 else 1

# Normalize θw and π across the entire dataset
inputs_array[:, -2] /= global_max_theta_w
inputs_array[:, -1] /= global_max_pi

# Shuffle and split the data
X_train_val, X_test, y_train_val, y_test, N_train_val, N_test = train_test_split(
    inputs_array, labels_array, population_sizes_array, test_size=0.15, random_state=42)

X_train, X_val, y_train, y_val, N_train, N_val = train_test_split(
    X_train_val, y_train_val, N_train_val, test_size=0.15 / 0.85, random_state=42)

print(f"Training samples: {X_train.shape[0]}")
print(f"Validation samples: {X_val.shape[0]}")
print(f"Test samples: {X_test.shape[0]}")

# Clean up
del inputs_array
del labels_array
gc.collect()

# Define the MLP model
input_shape = (NUM_FREQUENCY_BINS + 2,)  # Total number of features

model = Sequential([
    Dense(64, activation='relu', input_shape=input_shape),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(1)  # Output layer for regression
])

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

model.summary()

# Early stopping to prevent overfitting
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
val_dataset = val_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# Train the model
history = model.fit(
    train_dataset,
    epochs=100,
    validation_data=val_dataset,
    callbacks=[early_stopping]
)

# Evaluate the model on the test set
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

test_loss, test_mae = model.evaluate(test_dataset)
print(f"Test Loss (MSE): {test_loss}")
print(f"Test MAE: {test_mae}")

# Plot training & validation loss values
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], label='Training Loss (MSE)')
plt.plot(history.history['val_loss'], label='Validation Loss (MSE)')
plt.title('Model Loss')
plt.ylabel('Loss (MSE)')
plt.xlabel('Epoch')
plt.legend()
plt.show()

# Plot training & validation MAE values
plt.figure(figsize=(10, 5))
plt.plot(history.history['mae'], label='Training MAE')
plt.plot(history.history['val_mae'], label='Validation MAE')
plt.title('Model Mean Absolute Error')
plt.ylabel('MAE')
plt.xlabel('Epoch')
plt.legend()
plt.show()

# Make predictions on the test set
y_pred = model.predict(test_dataset)
y_pred = y_pred.flatten()

# Optional: Include population sizes and mutation rates for the test set
N_test = N_test[:len(y_test)]
mu_test = mutation_rates_array[-len(y_test):]

# Create a DataFrame with test results
test_results = pd.DataFrame({
    'True Theta': y_test,
    'Predicted Theta': y_pred,
    'Population Size': N_test,
    'Mutation Rate': mu_test
})

# Plot predicted vs. true theta values
plt.figure(figsize=(8, 8))
plt.scatter(test_results['True Theta'], test_results['Predicted Theta'], alpha=0.5)
plt.plot([test_results['True Theta'].min(), test_results['True Theta'].max()],
         [test_results['True Theta'].min(), test_results['True Theta'].max()], 'r--')
plt.xlabel('True Theta Values')
plt.ylabel('Predicted Theta Values')
plt.title('Predicted vs. True Theta Values')
plt.show()